In [14]:
import nltk
from nltk.corpus import brown
import pickle
import os
import nltk
import string
from nltk.corpus import brown
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import Word2Vec
import os
import tarfile
import urllib.request
import glob
from lxml import etree
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle
import functools
import operator
import os
import nltk
import string
from nltk.corpus import brown
from nltk.corpus import stopwords
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tqdm import tqdm
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

def preprocess_text(sentences):
    table = str.maketrans("", "", string.punctuation) # removing punctuation - it has to be a conscious decision
    tokenized_sentences = [[word.lower().translate(table) for word in sentence] for sentence in sentences]
    stop_words = set(stopwords.words("english"))
    tokenized_sentences = [[word for word in sentence if word and word not in stop_words] for sentence in tokenized_sentences]
    return tokenized_sentences

def save_tokenized_sentences(tokenized_sentences, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(tokenized_sentences, file)

def load_tokenized_sentences(file_path):
    with open(file_path, 'rb') as file:
        tokenized_sentences = pickle.load(file)
    return tokenized_sentences

def preprocess_text_unpacked(sentences):
    table = str.maketrans("", "", string.punctuation) # removing punctuation - it has to be a conscious decision
    tokenized_sentences = [[word.lower().translate(table) for word in sentence] for sentence in sentences]
    stop_words = set(stopwords.words("polish.txt"))
    tokenized_sentences = [[word for word in sentence if word and word not in stop_words] for sentence in tokenized_sentences]
    unpacked = functools.reduce(operator.iconcat, tokenized_sentences, [])
    return unpacked

def mapping(tokens):
    word_to_id = {}
    id_to_word = {}

    for i, token in enumerate(set(tokens)):
        word_to_id[token] = i
        id_to_word[i] = token

    return word_to_id, id_to_word

def generate_training_data(tokens, word_to_id, window_size):
    N = len(tokens)
    X, Y = [], []

    for i in range(N):
        nbr_inds = list(range(max(0, i - window_size), i)) + \
                   list(range(i + 1, min(N, i + window_size + 1)))
        for j in nbr_inds:
            X.append(word_to_id[tokens[i]])
            Y.append(word_to_id[tokens[j]])

    X = np.array(X)
    # X = np.expand_dims(X, axis=0)
    Y = np.array(Y)
    # Y = np.expand_dims(Y, axis=0)

    return X, Y

def batcherize(x_train, y_train, n_batches, n_batch):

    vocab_size = np.max(x_train) + 1
    dataset_length = x_train.shape[0]
    batch_size = int(np.floor(dataset_length / n_batches))

    y_train_batch = np.zeros((batch_size, vocab_size))
    y_train_batch[np.arange(batch_size), y_train[batch_size * n_batch : (n_batch + 1) * batch_size]] = 1

    x_train_batch = np.zeros((batch_size, vocab_size))
    x_train_batch[np.arange(batch_size), x_train[batch_size * n_batch : (n_batch + 1) * batch_size]] = 1

    return x_train_batch, y_train_batch

## Task 2

Implement the Word2Vec algorithm, using information provided during lecture. Test it on the brown and text8 corpora (corpuses?)

Next test the implementation on the Polish language corpus, check the correctness by checking token similiarity and vector arithmetics



POLISH DATASET -----------------------------------------------------

In [15]:
file_path = './polish_corpus/polish_tokenized_sentences.pkl'
polish_tokenized_sentences = load_tokenized_sentences(file_path)

preprocessed = preprocess_text_unpacked(polish_tokenized_sentences)
words, ids = mapping(preprocessed)

x_train, y_train = generate_training_data(preprocessed, words, 2)

In [9]:
tf.keras.utils.set_random_seed(2137)

model = tf.keras.Sequential()
model.add(Input(shape=(np.max(x_train) + 1, )))
model.add(Dense(50, activation='relu'))
model.add(Dense(np.max(x_train) + 1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 50)                6494100   
                                                                 
 dense_3 (Dense)             (None, 129881)            6623931   
                                                                 
Total params: 13,118,031
Trainable params: 13,118,031
Non-trainable params: 0
_________________________________________________________________


In [8]:
model = tf.keras.models.load_model('polish_keras_model')

n_batches = 5000
pbar = tqdm(range(0, n_batches))
for batch_idx in pbar:
    x_train_batch, y_train_batch = batcherize(x_train, y_train, n_batches, batch_idx)
    hist = model.fit(x_train_batch, y_train_batch, verbose=0, epochs=1)
    pbar.set_description(f"Latest accuracy: {hist.history['accuracy'][-1]}")

    if not batch_idx % 10:
        model.save('polish_keras_model')

Latest accuracy: 0.003937007859349251:  12%|█▏        | 7/57 [00:20<02:05,  2.51s/it] 

INFO:tensorflow:Assets written to: polish_keras_model\assets


INFO:tensorflow:Assets written to: polish_keras_model\assets
Latest accuracy: 0.0019685039296746254:  30%|██▉       | 17/57 [00:47<01:43,  2.58s/it]

INFO:tensorflow:Assets written to: polish_keras_model\assets


INFO:tensorflow:Assets written to: polish_keras_model\assets
Latest accuracy: 0.0019685039296746254:  47%|████▋     | 27/57 [01:14<01:14,  2.48s/it]

...

INFO:tensorflow:Assets written to: polish_keras_model\assets


INFO:tensorflow:Assets written to: polish_keras_model\assets
Latest accuracy: 0.011811023578047752:  82%|████████▏ | 47/57 [02:08<00:26,  2.63s/it] 

INFO:tensorflow:Assets written to: polish_keras_model\assets


INFO:tensorflow:Assets written to: polish_keras_model\assets
Latest accuracy: 0.011811023578047752:  88%|████████▊ | 50/57 [02:15<00:18,  2.70s/it]


IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (508,) (138,) 

In [67]:
preprocessed[280:290]

['zaakompaniować',
 'jedynej',
 'uczennicy',
 'papy',
 'moment',
 'musical',
 'szuberta',
 'jedynej',
 'uczennicy',
 'pierwszej']

In [23]:
model = tf.keras.models.load_model('polish_keras_model')
predict_id = words['jedynej']
test_sample = np.zeros((1, np.max(x_train)))
test_sample[0, predict_id] = 1

In [24]:
model_prediction = model.predict(test_sample)

1/1 [==============================] - 0s 49ms/step


In [25]:
prediction = np.argsort(model_prediction[0])[-5:]

In [26]:
sorted_descending_prediction = prediction[::-1]

In [27]:
sorted_descending_prediction

array([40456, 17036, 77998,  8574, 80629], dtype=int64)

In [28]:
for idx in sorted_descending_prediction:
    print(ids[idx])

opodal
serbię
kapłanów
uniwerku
poważniejszy
